# Scrapping Amazon Website for Book Information

<img src="books.jpg"
     alt="Books"
     style="height: 400px; margin-right: 150px;" 
     align="center"/>

<h3>Alexandre Rosseto Lemos</h3>
<h3>Date: June, 2021</h3>

## Problem Overview
This project uses Web Crawling and Web Scraping to retrieve information about books in the Amazon Kindle Store web page. It also uses Power BI as a visualization tool.

Using Python's libraries, I managed to obtain, for each book, all the information necessary (like number of pages, author, category) from Amazon web page and store it in a MS SQL database.

Then, using Power BI, I was able to create visualizations to help me gain insights of what I was reading.

In this project, I used the following Python libraries:

- selenium
- time
- bs4
- lxml
- pandas
- pymssql
- unidecode

At the end of this documentation, I provide the links to the official documentation pages of each library.

## Using Selenium to get to the correct book page

#### Selenium is a Web Crawling library, wich means I can use it to simulate a person navigating in the web. 

In this project, I used it to:
- Open Amazon's web page (using <a href="https://chromedriver.chromium.org/downloads">Google Chrome Web Driver</a>)
- Select the Kindle Store in the Search category
- Pass the name of the book and clicked in the "Search" button

Then, when the new page loaded, I used Selenium to select the first book in the list and click on the link, redirecting me to the book's page.

A book can have multiple versions, and each version has its own information, so it is important to obtain the information about the one I actually bought. In order to do so, I need to pick the first book that appears after searching for it using its name. The reason for this, is because I bought the versions that are on top of the "Most relevant" list, wich is the default sorting pattern used by Amazon's when displaying the search results.

In order to find the desired elements in the page, I had to manually inspect the front-end elements and find the ones necessary to use in finding the book's page.

#### bs4 and lxml are libraries that facilitate the manipulation of the information obtained in the front-end of web pages. 

In [1]:
# Initializing libraries
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import time
import bs4
import lxml
import pandas as pd
import pymssql
import unidecode

### Functions created for the code

In [2]:
def create_dict(book_inf, init_date, final_date):
    '''
    Info:
        This function transforms the list of informations into a dictionary
    -------------
    Input:
        book_inf: List of informations from the book (type: list)
        init_date: Date of the beginning of the reading (type: string)
        final_date: Date of the end of the reading (type: string)
    -------------
    Output:
        dict_info: Dictionary with the inforations from the book (type: dictionary)
    '''
    
    # Adding the dates where I began and finished reading the book
    dict_info = {}
    dict_info['Inicio_leitura'] = init_date
    dict_info['Fim_leitura'] = final_date
    
    # Index of the categories of the book
    i = 1
    for line in book_inf:
        
        # The categories in wich the book belongs to only have one element in the line. It's necessary to add the key to the
        # information
        try:
            formatted_name = unidecode.unidecode(line[0]).replace(' ', '_')  # Removes special characters and spaces
            dict_info[formatted_name]= line[1]
        except:
            
            # Limiting the maxium categories to 3
            if i <= 3:
                # Spliting the information into two fields: The category field and the category rank field
                cat_num, rank_num = 'Categoria_' + str(i), 'Ranking_Categoria_' + str(i)       # Creating the keys
                aux_split = line[0].split(' em ')     # Separating the rank information from the category information
                aux_split2 = aux_split[0].split(' ')  # Separating to get only the number
                dict_info[cat_num] = aux_split[1]     # Adding the category information to the dictionary

                for element in aux_split2:
                    try:
                        #if (i+1) <= 2:
                        dict_info[rank_num] = int(element)  # Ading the rank information to the dictionary
                        i += 1                              # A book can belong to many categories
                        #else:
                        #    pass
                    except:
                        pass
            else:
                pass
    
    return dict_info
    
def string_op(info_list):
    '''
    Info:
        This function makes string operations to clean and organize the information obtained from the book's web page
    -------------
    Input:
        info_list: List of informations from the book (type: list)
    -------------
    Output:
        formatted_list: Formatted list of informations from the book (type: list)
    '''
        
    # First operation: I only want the rank of the book in the Kindle Store, and usually this information is mixed with
    # other things. So I only take what is before the first '(' in the string. Also, I just want the numeric value of the field.
    for rank_info in info_list:
        if rank_info[0] == 'Ranking dos mais vendidos':    # Searching for the correct field
            rank_info[1] = rank_info[1].split('(')[0].split(' ')[1]      # Replacing the information
        
        # Second operation: I only want the numeric value of the field that has the number of pages
        if rank_info[0] == 'Número de páginas':
            rank_info[1] = rank_info[1].split(' ')[0]
            
        # Third operation: Obtaining the publisher
        if rank_info[0] == 'Editora':
            rank_info[1] = rank_info[1].split(';')[0]
            try:
                rank_info[1] = rank_info[1].split('(')[0]
            except:
                pass

    # Fourth operation: The star ranking comes mixed with the amount of classifications made by the customers.
    # To get split the informations, I edit the string and then split it.
    # Obs.: these informations are always in the last position of the list
    star_infos = info_list[-1][1].replace('estrelas', 'estrelas|').split('|')
    stars_users = star_infos[0].split(' de ')[0]    # Amount of stars given by the users
    total_stars = star_infos[0].split(' de ')[1].split(' ')[0]                         # Maximum amount of stars possible
    info_list[-1][1] = stars_users                                     # Replacing the mixed informations with the number of stars
    info_list.append(['Avaliacao Maxima', total_stars])                # Adding the maxium score possible
    info_list.append(['Classificacoes', star_infos[1].split(' ')[0]])  # Adding the amount of classifications made by the customers
    
    
    
    return info_list

def get_book_info(book_name, lang):

    '''
    Info:
        This function uses web crawl and web scrape to navigate to the book's Amazon web page and obtain the desired 
        informations about it.
    -------------
    Input:
        - book_name: Name of the book to get the information (type: str)
        - lang: Language of the book
    -------------
    Output:
        - book_info: List containing the informations about the book (type: list)
    '''
    
    # Using the 'headless' argument to avoid the chrome popping up
    op = webdriver.ChromeOptions()
    op.add_argument('headless')

    # Passing the path to the chromedriver.exe file and the url to the Amazon page
    CHROME_DRIVER_PATH = "PATH TO THE DRIVER/chromedriver.exe"  # Change to the path to the chromedriver.exe in your machine
    
    amazon_url = "https://www.amazon.com.br/"

    # Start of the web crawl
    driver = webdriver.Chrome(CHROME_DRIVER_PATH,
                              options=op
                              )
    driver.get(amazon_url)

    # Selecting the 'Loja Kindle' option to use in the search.
    category = Select(driver.find_element_by_id("searchDropdownBox"))    # Searching for the DropDown box       
    category.select_by_visible_text("Loja Kindle")                       # Selecting the desired section

    # Passing the name of the book to the search bar
    search_var = book_name + ' ' + lang                                   # Adding the language of the book
    searchbar = driver.find_element_by_id("twotabsearchtextbox")         # Searching for the Search bar by id
    searchbar.send_keys(search_var, Keys.TAB, Keys.ENTER)                # Passing the book's title and pressing the search button

    # Waiting for the new page to load
    time.sleep(3)
    
    # Ordering the results by the users score
    sorting = Select(driver.find_element_by_id("s-result-sort-select"))
    sorting.select_by_value("review-rank")
    # Waiting for the page to load
    time.sleep(2)

    # Obtaining all the books in the page
    books = driver.find_elements_by_xpath("//a[@class = 'a-link-normal a-text-normal']")

    # Checking for the book with the same name as given the one given
    unmatching_names = True
    ind = 0  # Book index in the list
    while unmatching_names:

        # Retrieving the name of the book in the web page
        web_name = books[ind].find_elements_by_xpath(".//span")
        book_web_name = web_name[0].text

        # If the names match, select the book
        if book_web_name == book_name:
            unmatching_names = False
            books[ind].click()    # Selecting it

        # If the names don't match, search for the next element in the list
        else:
            ind += 1
    
    # Waiting for the new page to load
    time.sleep(4)

    # Obtaining the html code for the book page
    book_page = driver.page_source
    
    #Closing google Chrome
    driver.quit()

    # Using the bs4 and lxml libraries to work with the page source code
    soup = bs4.BeautifulSoup(book_page,"lxml")

    # Obtaining the desired information from the page
    # Finding the div elements 
    div_el = soup.select("div")

    book_info = []
    for info in div_el: 
        try:
            # The desired information is located in the element with the id = 'detailBullets_feature_div'
            if info['id'] == 'detailBullets_feature_div':
                book_info.append(info)
        except:
            pass

    # Obtaining the price of the book
    try:
        price_ele = soup.find_all('span', {'id': "kindle-price"})
        book_price = price_ele[0].text.replace('\n', '').split('$')[1]
        book_price= book_price.replace(' ','')
    except:
        price_ele = soup.find_all('span', {'class': "slot-price"})
        book_price = price_ele[0].text.replace('\n', '').split('$')[1]
        book_price= book_price.replace(' ','')

    list_book_info = [['Livro', book_name], ['Preço (R$)', book_price]]   # Adding the book name to the list of informations
    for line in book_info[0].select('li'):
        list_book_info.append(line.text.replace('\n','').split(':'))
        
    formatted_book_info = string_op(list_book_info)
    
    return formatted_book_info

### Running the code

In [3]:
# Example of a list of books. The structure is:
# ('Name of the book', 'Language', 'Date of the beggining of the reading', 'Date of the ending of the reading (if exists)')
books = [('O andar do bêbado: Como o acaso determina nossas vidas', 'Português', '06/02/2021', '02/03/2021'), 
         ('A revolução dos bichos', 'Português', '02/03/2021', '04/03/2021'),
         ('1984', 'Português', '29/03/2021', '17/05/2021'),
         ('101 Perguntas e Respostas para Investidores Iniciantes', 'Português','10/01/2021', '04/02/2021'),
         ('101 Perguntas E Respostas Sobre Tributação Em Renda Variável: Tire suas dúvidas sobre tributação para Bolsa de Valores', 'Português','09/03/2021', 'Inacabado'),
         ('Rápido e devagar: Duas formas de pensar', 'Português','18/05/2021', 'Inacabado'),
         ('Memórias Póstumas de Brás Cubas (Prazer de Ler)', 'Português', '01/02/2021', '10/02/2021'),
         ('Hábitos Atômicos: Um Método Fácil e Comprovado de Criar Bons Hábitos e Se Livrar dos Maus.','Português', '03/01/2021', 'Inacabado'),
         ('O Banquete (o amor, o belo)', 'Português', '15/01/2021', '30/03/2021'),
         ('Homo Deus: Uma breve história do amanhã','Português', '15/02/2021', '20/02/2021'),
         ('Bíblia Sagrada', 'Português', '01/01/2021', '06/05/2021'),
        ]

# Obtaining the information about the books and storing it in a Pandas DataFrame structure
books_info = pd.DataFrame()
for book in books:
    book_info = get_book_info(book[0],book[1])
    book_info_dict = create_dict(book_info, book[2], book[3])
    books_info = books_info.append(book_info_dict, ignore_index=True)


In [4]:
books_info.head()

,ASIN,Avaliacao_Maxima,Avaliacoes_dos_clientes,Categoria_1,Categoria_2,Classificacoes,Configuracao_de_fonte,Dicas_de_vocabulario,Editora,Fim_leitura,...,Numero_de_paginas,Preco_(R$),Ranking_Categoria_1,Ranking_Categoria_2,Ranking_dos_mais_vendidos,Tamanho_do_arquivo,Leitor_de_tela,Categoria_3,Ranking_Categoria_3,Quantidade_de_dispositivos_em_que_e_possivel_ler_este_eBook_ao_mesmo_tempo
0,B008FPZPRA,5,"4,6",Probabilidade e Estatística,"Ciências, Matemática e Tecnologia",1.038,Habilitado,Não habilitado,Zahar,02/03/2021,...,270,"29,90",3.0,16.0,"1,682",787 KB,NaN,NaN,NaN,NaN
1,B009WWDBX0,5,"4,9",Ficção clássica,Clássicos de Ficção,17.557,Habilitado,Não habilitado,Companhia das Letras,04/03/2021,...,156,"19,90",13.0,49.0,643,1285 KB,NaN,NaN,NaN,NaN
2,B009XE662U,5,"4,9",Ficção clássica,Clássicos de Ficção,13.572,Habilitado,Não habilitado,Companhia das Letras,17/05/2021,...,482,"29,90",17.0,66.0,883,2686 KB,Compatível,NaN,NaN,NaN
3,B07L8NR3DF,5,"4,7",Equipes,Investir,4.101,Habilitado,Não habilitado,Suno Research,04/02/2021,...,109,"20,00",9.0,23.0,"1,119",218 KB,Compatível,Negócios e economia,36.0,NaN
4,B07Q4SN446,5,"4,5",Financeiro,Investir,655,Habilitado,Não habilitado,NaN,Inacabado,...,99,"20,00",7.0,75.0,"6,589",3335 KB,NaN,Negócios e economia,294.0,NaN


### Selecting the desired information

In [5]:
# All the columns obtained
original_columns = books_info.columns
original_columns

Index(['ASIN', 'Avaliacao_Maxima', 'Avaliacoes_dos_clientes', 'Categoria_1',
       'Categoria_2', 'Classificacoes', 'Configuracao_de_fonte',
       'Dicas_de_vocabulario', 'Editora', 'Fim_leitura', 'Idioma',
       'Inicio_leitura', 'Leitura_de_texto', 'Livro', 'Numero_de_paginas',
       'Preco_(R$)', 'Ranking_Categoria_1', 'Ranking_Categoria_2',
       'Ranking_dos_mais_vendidos', 'Tamanho_do_arquivo', 'Leitor_de_tela',
       'Categoria_3', 'Ranking_Categoria_3',
       'Quantidade_de_dispositivos_em_que_e_possivel_ler_este_eBook_ao_mesmo_tempo'],
      dtype='object')

In [6]:
# List of desired information
desired_columns = ['ASIN',                      # Amazon code for the book
                   'Livro',                     # Name of the book
                   'Editora',                   # Publisher
                   'Numero_de_paginas',         # Number of pages
                   'Idioma',                    # Language
                   'Preco_(R$)',                # Price
                   'Avaliacao_Maxima',          # Maixum possible score
                   'Avaliacoes_dos_clientes',   # User score
                   'Classificacoes',            # Number of avaluations made by the users
                   'Categoria_1',               # Book category 1
                   'Categoria_2',               # Book category 2
                   'Categoria_3',               # Book category 3
                   'Ranking_Categoria_1',       # Ranking in category 1
                   'Ranking_Categoria_2',       # Ranking in category 2
                   'Ranking_Categoria_3',       # Ranking in category 3
                   'Inicio_leitura',            # Date of the begin of the reading
                   'Fim_leitura'                # Date of the end of the reading
                  ]

# Creating the DataFrame with the desired informations
desired_books_info = books_info[desired_columns]

In [7]:
#desired_books_info.head()
desired_books_info = desired_books_info.fillna('Não possui')   # Replacing the NaN values
desired_books_info

,ASIN,Livro,Editora,Numero_de_paginas,Idioma,Preco_(R$),Avaliacao_Maxima,Avaliacoes_dos_clientes,Classificacoes,Categoria_1,Categoria_2,Categoria_3,Ranking_Categoria_1,Ranking_Categoria_2,Ranking_Categoria_3,Inicio_leitura,Fim_leitura
0,B008FPZPRA,O andar do bêbado: Como o acaso determina noss...,Zahar,270,Português,"29,90",5,"4,6",1.038,Probabilidade e Estatística,"Ciências, Matemática e Tecnologia",Não possui,3.0,16.0,Não possui,06/02/2021,02/03/2021
1,B009WWDBX0,A revolução dos bichos,Companhia das Letras,156,Português,"19,90",5,"4,9",17.557,Ficção clássica,Clássicos de Ficção,Não possui,13.0,49.0,Não possui,02/03/2021,04/03/2021
2,B009XE662U,1984,Companhia das Letras,482,Português,"29,90",5,"4,9",13.572,Ficção clássica,Clássicos de Ficção,Não possui,17.0,66.0,Não possui,29/03/2021,17/05/2021
3,B07L8NR3DF,101 Perguntas e Respostas para Investidores In...,Suno Research,109,Português,"20,00",5,"4,7",4.101,Equipes,Investir,Negócios e economia,9.0,23.0,36.0,10/01/2021,04/02/2021
4,B07Q4SN446,101 Perguntas E Respostas Sobre Tributação Em ...,Não possui,99,Português,"20,00",5,"4,5",655,Financeiro,Investir,Negócios e economia,7.0,75.0,294.0,09/03/2021,Inacabado
5,B00A3D1A44,Rápido e devagar: Duas formas de pensar,Objetiva,641,Português,"29,90",5,"4,7",4.376,Tomar de Decisões e Resolução de Problemas,Negócios e economia,Não possui,8.0,18.0,Não possui,18/05/2021,Inacabado
6,B07RLDSL77,Memórias Póstumas de Brás Cubas (Prazer de Ler),Edições Câmara,388,Português,"0,00",5,"4,7",4.663,Ficção clássica,Não possui,Não possui,3.0,Não possui,Não possui,01/02/2021,10/02/2021
7,B07V8LHWZ5,Hábitos Atômicos: Um Método Fácil e Comprovado...,Alta Books,302,Português,"16,11",5,"4,8",2.362,Autoajuda e desenvolvimento pessoal,Autoajuda,Não possui,12.0,42.0,Não possui,03/01/2021,Inacabado
8,B00AGAP3DS,"O Banquete (o amor, o belo)",Não possui,Não possui,Português,"0,00",5,"4,6",1.662,Filosofia,Não possui,Não possui,3.0,Não possui,Não possui,15/01/2021,30/03/2021
9,B01LXCJTO4,Homo Deus: Uma breve história do amanhã,Companhia das Letras,476,Português,"37,90",5,"4,8",4.820,História e Filosofia da Ciência,"Ciências, Matemática e Tecnologia",Não possui,6.0,20.0,Não possui,15/02/2021,20/02/2021


### Functions created to pass queries to the MS SQL database

In [8]:
def create_table_sql(cursor, columns , info):
    '''
    Info:
        Function used to create a table in the MS SQL server if it doesn't exists alredy
    -------------
    Input:
        cursor: connection cursor created for the database (type: pymssql.Cursor)
        columns: desired columns of the dataframe (type: list)
        info: dataframe with the desired information (type: pandas DataFrame)
     -------------
     Output:
         desired_columns_type: list with the types of the data from the dataframe (type: list)
    '''
    
    # Creating the list of data types to use in the creation of the table
    desired_columns_type = []
    for info_type in info.dtypes:
            desired_columns_type.append('VARCHAR(300)')
            
    # Creating the sql command used to create the new table into the database if it doesn't exists alredy
    sql_create_table = "IF OBJECT_ID('Book_Info', 'U') IS NULL CREATE TABLE Book_Info ("
    for ind_inf, inf in enumerate(desired_columns):
        sql_create_table = sql_create_table + f'"{inf}" {desired_columns_type[ind_inf]},'
    sql_create_table = sql_create_table + ' PRIMARY KEY("ASIN"))'
            
    # Creating the SQL table
    cursor.execute(sql_create_table)
    conn.commit()
    
 
def insert_into_table_sql(cursor, books_info):
    '''
    Info:
        This function inserts the information of the books to the table in the MS SQL server
    -------------
    Input:
        cursor: connection cursor created for the database (type: pymssql.Cursor)
        books_info: dataframe with the desired information (type: pandas DataFrame)
    -------------
    Output:
        None
    '''
    
    # Insert Dataframe into SQL Server:
    for index, row in books_info.iterrows():
        try:
            cursor.execute(
                "INSERT INTO dbo.Book_Info VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", 
                (row['ASIN'], row['Livro'], row['Editora'], row['Numero_de_paginas'], row['Idioma'], row['Preco_(R$)'], 
                 row['Avaliacao_Maxima'], row['Avaliacoes_dos_clientes'], row['Classificacoes'], row['Categoria_1'], row['Categoria_2'], 
                 row['Categoria_3'], row['Ranking_Categoria_1'], row['Ranking_Categoria_2'], row['Ranking_Categoria_3'], 
                 row['Inicio_leitura'], row['Fim_leitura']
                )
            )
        except:
            pass
    
    conn.commit()

In [9]:
# Setting the connection information
server = 'SERVER NAME'
user = 'USER'
password = 'PASSWORD'
conn = pymssql.connect(server, user, password, "DATABASE TABLE")

# Creating the cursor
cursor = conn.cursor()

# Creating the table
#list_type = 
create_table_sql(cursor, desired_columns, desired_books_info)

# Inserting data into the table
insert_into_table_sql(cursor, desired_books_info)

# Printing the first row in the table
cursor.execute("SELECT TOP 1 * FROM Book_Info")
row = cursor.fetchone()
while row:
    print(row)
    row = cursor.fetchone()

# Closing the connection
conn.close()

('B008FPZPRA', 'O andar do bêbado: Como o acaso determina nossas vidas', 'Zahar', '270', 'Português', '29,90', '5', '4,6', '1.038', 'Probabilidade e Estatística', 'Ciências, Matemática e Tecnologia', 'Não possui', '3.0', '16.0', 'Não possui', '06/02/2021', '02/03/2021')


## Libraries documentation/main pages:
- <a href="https://selenium-python.readthedocs.io/index.html">Selenium</a>
- <a href="https://pandas.pydata.org">Pandas</a>
- <a href="https://docs.python.org/3/library/time.html">Time</a>
- <a href="https://www.crummy.com/software/BeautifulSoup/bs4/doc/">Bs4</a>
- <a href="https://lxml.de/">Lxml</a>
- <a href="https://pypi.org/project/pymssql/">PyMSSQL</a>
- <a href="https://pypi.org/project/Unidecode/">Unidecode</a>